In [1]:
import time as tm
# import predict
import numpy as np
import cv2
from sklearn.linear_model import LogisticRegression

# from sklearn.neighbors import KNeighborsClassifier
# from google.colab.patches import cv2_imshow

In [2]:
def extract_background_color(image):
    height, width, _ = image.shape

    tl = image[0, 0]
    tr = image[0, width - 1]
    bl = image[height - 1, 0]
    br = image[height - 1, width - 1]

    if (np.array_equal(tl, tr)) and (np.array_equal(tl, bl)) and (np.array_equal(tl, br)):
        return tl
    elif ( ((np.array_equal(tl, tr)) and (np.array_equal(tl, bl))) or ((np.array_equal(tl, bl)) and (np.array_equal(tl, br))) or ((np.array_equal(tl, tr)) and (np.array_equal(tl, br))) ):
        return tl
    elif ( ((np.array_equal(tr, tl)) and (np.array_equal(tr, bl))) or ((np.array_equal(tr, bl)) and (np.array_equal(tr, br))) or ((np.array_equal(tr, tl)) and (np.array_equal(tr, br))) ):
        return tr
    elif ( ((np.array_equal(bl, tr)) and (np.array_equal(bl, tl))) or ((np.array_equal(bl, tl)) and (np.array_equal(bl, br))) or ((np.array_equal(bl, tr)) and (np.array_equal(bl, br))) ):
        return bl
    elif ( ((np.array_equal(br, tl)) and (np.array_equal(br, bl))) or ((np.array_equal(br, bl)) and (np.array_equal(br, tr))) or ((np.array_equal(br, tl)) and (np.array_equal(br, tr))) ):
        return br
    elif (np.array_equal(tr, br) or np.array_equal(tr, bl) or np.array_equal(tr, tl)):
        return tr
    elif (np.array_equal(tl, br) or np.array_equal(tl, bl) or np.array_equal(tr, tl)):
        return tl
    elif (np.array_equal(br, tr) or np.array_equal(br, bl) or np.array_equal(br, tl)):
        return br
    else:
        return bl

In [ ]:
def trainDecaptcha(filenames):
    X_train = np.zeros((len(filenames), 100 * 100))
    y_train = np.loadtxt("labels.txt", dtype=str)
    y_train = y_train[:len(filenames)]

    for i, file in enumerate(filenames):
        img = cv2.imread(file)
        # cv2_imshow(img)
        img = img[:, 358:458]
        # cv2_imshow(img)

        hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        indices = np.where((hsv_img == extract_background_color(hsv_img)).all(axis=2))
        hsv_img[indices] = [255, 255, 255]

        kernel = np.ones((5, 5), np.uint8)
        kernel1 = np.ones((3, 3), np.uint8)
        dilate_img = cv2.dilate(hsv_img, kernel, iterations=1)
        gray_img = cv2.cvtColor(dilate_img, cv2.COLOR_BGR2GRAY)
        gray_img = cv2.fastNlMeansDenoising(gray_img, None, 20, 7, 21)
        erod_img = cv2.erode(gray_img, kernel1, iterations=1)
        _, otsu_img = cv2.threshold(erod_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        cv2_imshow(otsu_img)
        otsu = otsu_img / 255.
        otsu = np.array(otsu)
        otsu = otsu.flatten()
        X_train[i] = otsu

    X_train = np.array(X_train)
    model = LogisticRegression(max_iter=100)
    model.fit(X_train, y_train)
    return model

train_size = 10
train_file = [ "/content/drive/MyDrive/Colab/train/%d.png" % i for i in range( train_size ) ]
model = trainDecaptcha(train_file)

def decaptcha(filenames):
    X_train = np.zeros((len(filenames), 100 * 100))
    for i, file in enumerate(filenames):
        img = cv2.imread(file)
        img = img[:, 358:458]
        hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        indices = np.where((hsv_img == extract_background_color(hsv_img)).all(axis=2))
        hsv_img[indices] = [255, 255, 255]
        kernel = np.ones((5, 5), np.uint8)
        kernel1 = np.ones((3, 3), np.uint8)
        dilate_img = cv2.dilate(hsv_img, kernel, iterations=1)
        gray_img = cv2.cvtColor(dilate_img, cv2.COLOR_BGR2GRAY)
        gray_img = cv2.fastNlMeansDenoising(gray_img, None, 20, 7, 21)
        erod_img = cv2.erode(gray_img, kernel1, iterations=1)
        _, otsu_img = cv2.threshold(erod_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        otsu = otsu_img / 255.
        otsu = np.array(otsu)
        otsu = otsu.flatten()
        X_train[i] = otsu

    X_train = np.array(X_train)
    y_pred = model.predict(X_train)
    return y_pred

In [ ]:
def rotate_img(file, angle):
    tmp = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    tmp = cv2.bitwise_not(tmp)
    rot = cv2.getRotationMatrix2D((100 / 2, 100 / 2), angle, 1.0)
    tmp = cv2.warpAffine(tmp, rot, (100, 100))
    tmp = cv2.bitwise_not(tmp)
    return tmp

In [ ]:
import time as tm
from PIL import Image
import pickle

def decaptcha2(filenames):
    l = []
    # ref_images = []
    # tstfile = ["/content/drive/MyDrive/Colab/reference/%d.png" % i for i in {0,2,4,6,8,10,12,14}]
    # for file in tstfile:
    #     tmp = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    #     ref_images.append(tmp)
    #     for angle in {10,-10,20,-20,30,-30}:
    #         ref_images.append(rotate_img(file,angle))
    # tstfile = ["/content/drive/MyDrive/Colab/reference/%d.png" % i for i in {1,3,5,7,9,11,13,15}]
    # for file in tstfile:
    #     tmp = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    #     ref_images.append(tmp)
    #     for angle in {10,-10,20,-20,30,-30}:
    #         ref_images.append(rotate_img(file,angle))
    # with open('ref_images.pkl', 'wb') as file:
    #     pickle.dump(ref_images, file)

    with open('ref_images.pkl', 'rb') as file:
        ref_images = pickle.load(file)

    for i, file in enumerate(filenames):
        img = Image.open(file)  # Open the image
        img = np.array(img)
        # img = cv2.imread(file, cv2.IMREAD_UNCHANGED)
        img = img[:, 358:458]
        indices = np.where((img == extract_background_color(img)).all(axis=2))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img[indices] = [255]
        # _, otsu_img = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY)
        otsu_img = img
        cropped_bottom = otsu_img[:5, :]
        cropped_top = otsu_img[5:, :]
        otsu_img = np.concatenate((cropped_top, cropped_bottom), axis=0)
        # cv2_imshow(otsu_img)
        ans = (900000, -1)
        for j,image in enumerate(ref_images):
            bitwise_xor = cv2.bitwise_xor(image, otsu_img)
            tmpvar = cv2.sumElems(bitwise_xor)[0]
            if(tmpvar<ans[0]):
                ans = (tmpvar,j)
        if(ans[1]>55):
            l.append("ODD")
        else:
            l.append("EVEN")

    return l


tss = 1500
tse = 1510
train_file2 = ["/content/drive/MyDrive/Colab/train/%d.png" % i for i in range(tss,tse)]

tic = tm.perf_counter()
output = decaptcha2(train_file2)
toc = tm.perf_counter()
print((toc - tic) / (tse-tss))
print(output)

0.005398607600000105
['EVEN', 'EVEN', 'EVEN', 'EVEN', 'ODD', 'ODD', 'EVEN', 'ODD', 'ODD', 'EVEN']


In [ ]:
#################  METHOD 3  #################
from joblib import dump, load
import skimage.io as io

def trainDecaptcha3(filenames):
    X_train = np.zeros((len(filenames), 100 * 100))
    y_train = np.loadtxt("/content/drive/MyDrive/Colab/train/labels.txt", dtype=str)
    y_train = y_train[:len(filenames)]

    for i, file in enumerate(filenames):
        img = cv2.imread(file)
        img = img[:, 358:458]
        indices = np.where((img == extract_background_color(img)).all(axis=2))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img[indices] = [255]
        _, otsu_img = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY)
        # cropped_bottom = np.full((5, 100), 255, dtype=np.uint8)
        # cropped_top = otsu_img[5:, :]
        # otsu_img = np.concatenate((cropped_top, cropped_bottom), axis=0)
        # cv2_imshow(otsu_img)
        otsu = otsu_img / 255.
        otsu = np.array(otsu)
        otsu = otsu.flatten()
        X_train[i] = otsu

    model = LogisticRegression(max_iter=100)
    model.fit(X_train, y_train)

    dump(model, 'model.joblib')

    return model

train_size = 500
train_file = [ "/content/drive/MyDrive/Colab/train/%d.png" % i for i in range( train_size ) ]
model = trainDecaptcha3(train_file)

In [ ]:
def decaptcha3(filenames):
    X_train = np.zeros((len(filenames), 100 * 100))
    for i, file in enumerate(filenames):
        img = cv2.imread(file)
        img = img[:, 358:458]
        indices = np.where((img == extract_background_color(img)).all(axis=2))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img[indices] = [255]
        _, otsu_img = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY)
        otsu = otsu_img / 255.
        otsu = np.array(otsu)
        otsu = otsu.flatten()
        X_train[i] = otsu

    # model = LogisticRegression()
    model = load('model.joblib')
    y_pred = model.predict(X_train)
    return y_pred


tss = 50
tse = 100
tff = ["/content/drive/MyDrive/Colab/train/%d.png" % i for i in range(tss,tse)]

tic = tm.perf_counter()
prediction = decaptcha3(tff)
toc = tm.perf_counter()
print((toc - tic) / (tse-tss))

print(prediction)

0.0041815361000044505
['ODD' 'EVEN' 'EVEN' 'ODD' 'EVEN' 'EVEN' 'EVEN' 'ODD' 'EVEN' 'EVEN' 'ODD'
 'ODD' 'ODD' 'ODD' 'ODD' 'EVEN' 'ODD' 'EVEN' 'EVEN' 'EVEN' 'EVEN' 'EVEN'
 'ODD' 'EVEN' 'EVEN' 'EVEN' 'EVEN' 'ODD' 'ODD' 'EVEN' 'ODD' 'ODD' 'EVEN'
 'EVEN' 'ODD' 'EVEN' 'ODD' 'EVEN' 'EVEN' 'ODD' 'ODD' 'ODD' 'ODD' 'ODD'
 'ODD' 'ODD' 'ODD' 'EVEN' 'ODD' 'EVEN']


In [ ]:
#################  METHOD 4  #################
import concurrent.futures

def trainDecaptcha4(filenames):
    X_train = np.zeros((len(filenames), 100 * 100))
    y_train = np.loadtxt("/content/drive/MyDrive/Colab/train/labels.txt", dtype=str)
    y_train = y_train[:len(filenames)]

    for i, file in enumerate(filenames):
        img = cv2.imread(file)
        img = img[:, 358:458]
        indices = np.where((img == extract_background_color(img)).all(axis=2))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img[indices] = [255]
        _, otsu_img = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY)
        # cropped_bottom = np.full((5, 100), 255, dtype=np.uint8)
        # cropped_top = otsu_img[5:, :]
        # otsu_img = np.concatenate((cropped_top, cropped_bottom), axis=0)
        # cv2_imshow(otsu_img)
        otsu = otsu_img / 255.
        otsu = np.array(otsu)
        otsu = otsu.flatten()
        X_train[i] = otsu

    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X_train, y_train)
    dump(knn, 'knn.joblib')
    return knn

train_size = 2000
train_file = [ "/content/drive/MyDrive/Colab/train/%d.png" % i for i in range( train_size ) ]
knn2 = trainDecaptcha4(train_file)

def decaptcha4(filenames):
    X_train = np.zeros((len(filenames), 100 * 100))
    for i, file in enumerate(filenames):
        img = cv2.imread(file)
        img = img[:, 358:458]
        indices = (img == extract_background_color(img)).all(axis=2)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img[indices] = [255]
        _, otsu_img = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY)
        # cropped_bottom = np.full((5, 100), 255, dtype=np.uint8)
        # cropped_top = otsu_img[5:, :]
        # otsu_img = np.concatenate((cropped_top, cropped_bottom), axis=0)

        otsu = otsu_img / 255.
        otsu = np.array(otsu)
        otsu = otsu.flatten()
        X_train[i] = otsu

    knn = load('knn.joblib')
    y_pred = knn.predict(X_train)
    return y_pred


tss = 0
tse = 100
tff = ["/content/drive/MyDrive/Colab/train/%d.png" % i for i in range(tss,tse)]

tic = tm.perf_counter()
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     prediction = executor.map(decaptcha3, tff)
prediction = decaptcha3(tff)
toc = tm.perf_counter()
print((toc - tic) / (tse-tss))



0.00614365772000383


In [ ]:
from sklearn.linear_model import LinearRegression

def trainDecaptcha5(filenames):
    X_train = np.zeros((len(filenames), 100 * 100))
    y_train = np.loadtxt("/content/drive/MyDrive/Colab/train/labels.txt", dtype=str)
    y_train = y_train[:len(filenames)]
    y_train = np.where(y_train == "EVEN", 0, 1)

    for i, file in enumerate(filenames):
        img = cv2.imread(file)
        img = img[:, 358:458]
        indices = (img == extract_background_color(img)).all(axis=2)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img[indices] = [255]
        _, otsu_img = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY)
        otsu = otsu_img / 255.
        otsu = np.array(otsu)
        otsu = otsu.flatten()
        X_train[i] = otsu
    model = LinearRegression()
    model.fit(X_train, y_train)
    dump(model, 'linearmodel.joblib')
    return model

train_size = 2000
train_file = [ "/content/drive/MyDrive/Colab/train/%d.png" % i for i in range( train_size ) ]
linerr = trainDecaptcha5(train_file)

def decaptcha5(filenames):
    X_train = np.zeros((len(filenames), 100 * 100))
    for i, file in enumerate(filenames):
        img = cv2.imread(file)
        img = img[:, 358:458]
        indices = (img == extract_background_color(img)).all(axis=2)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img[indices] = [255]
        _, otsu_img = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY)
        otsu = otsu_img / 255.
        otsu = np.array(otsu)
        otsu = otsu.flatten()
        X_train[i] = otsu
    linearmodel = load('linearmodel.joblib')
    y_pred = linearmodel.predict(X_train)
    return y_pred


tss = 0
tse = 100
tff = ["/content/drive/MyDrive/Colab/train/%d.png" % i for i in range(tss,tse)]

tic = tm.perf_counter()
prediction = decaptcha5(tff)
toc = tm.perf_counter()
print((toc - tic) / (tse-tss))

0.004304970659995888


In [ ]:
import numpy as np
import cv2
import pickle

def trainDecaptcha3(filenames):
    X_train = np.zeros((len(filenames), 100 * 100))
    y_train = np.loadtxt("/content/drive/MyDrive/Colab/train/labels.txt", dtype=str)
    y_train = y_train[:len(filenames)]

    for i, file in enumerate(filenames):
        img = cv2.imread(file)
        img = img[:, 358:458]
        indices = np.where((img == extract_background_color(img)).all(axis=2))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img[indices] = [255]
        _, otsu_img = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY)
        otsu = otsu_img / 255.
        otsu = np.array(otsu)
        otsu = otsu.flatten()
        X_train[i] = otsu

    model = Sequential([
        Conv2D(32, (3,3), activation = 'relu', input_shape = (100, 100)),
        MaxPooling2D((2,2)),

        Conv2D(32, (3,3), activation = 'relu'),
        MaxPooling2D((2,2)),

        Flatten(),
        Dense(64, activation = 'relu'),
        Dense(1, activation = 'sigmoid')
    ])
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.fit(X_train, y_train, epochs = 5, batch_size = 64)


    # model = LogisticRegression()
    # model.train(X_train, y_train)

    # Save the model to a file
    # with open('model.pkl', 'wb') as file:
    #     pickle.dump(model, file)
    return model

train_size = 200
train_file = [ "/content/drive/MyDrive/Colab/train/%d.png" % i for i in range( train_size ) ]
model = trainDecaptcha3(train_file)


In [ ]:
import time as tm
num_test = 2000
filepaths = [ "/content/drive/MyDrive/Colab/train/%d.png" % i for i in range(num_test ) ]
file = open( "/content/drive/MyDrive/Colab/train/labels.txt", "r" )
gold_output = file.read().splitlines()
file.close()

# Get recommendations from predict.py and time the thing
tic = tm.perf_counter()
output = decaptcha3( filepaths )
toc = tm.perf_counter()

parity_match = np.array( [ 1 if x.strip().upper() == y.strip().upper() else 0 for ( x, y ) in zip ( output, gold_output ) ] ).sum()

print( f"Time taken per image is {(toc - tic) / num_test} seconds" )
print( f"Parity match score is {parity_match / num_test}" )